<a href="https://colab.research.google.com/github/MohammadMadadi94/Rahnema-ML/blob/main/MovieLenzCoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Movie lenz 1M dataset

Download, Loading and unzipping dataset

In [2]:
!wget https://files.grouplens.org/datasets/movielens/ml-1m.zip

--2022-02-21 21:18:48--  https://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip’

ml-1m.zip           100%[===================>]   5.64M  11.9MB/s    in 0.5s    

2022-02-21 21:18:48 (11.9 MB/s) - ‘ml-1m.zip’ saved [5917549/5917549]



In [3]:
!unzip ml-1m.zip

Archive:  ml-1m.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         


In [4]:
import pandas as pd

In [5]:
movies = pd.read_csv('ml-1m/movies.dat', sep = '::' , header = None, names = ['movieId', 'title', 'genre'],  encoding='latin-1')

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [6]:
movies.head()

,movieId,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
users = pd.read_csv('ml-1m/users.dat', sep = '::' , header = None, names = ['UserID','Gender', 'Age', 'Occupation', 'Zip-code'],  encoding='latin-1')

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [8]:
users.head()


,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [9]:
ratings = pd.read_csv('ml-1m/ratings.dat', sep = '::' , header = None, names = ['UserID', 'MovieID', 'Rating','Timestamp'],  encoding='latin-1')

/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


In [10]:
ratings.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


feature engineering data (FED)

In [11]:
from sklearn.preprocessing import MultiLabelBinarizer

In [12]:
movies.genre = movies.genre.str.split('|')

In [13]:
mlb = MultiLabelBinarizer()

In [14]:
mlb.fit(movies.genre)

MultiLabelBinarizer()

In [15]:
mlb.transform(movies.genre)

array([[0, 0, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0]])

In [16]:
mlb.classes_

array(['Action', 'Adventure', 'Animation', "Children's", 'Comedy',
       'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
       'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War',
       'Western'], dtype=object)

In [17]:
movies_genre = pd.DataFrame(mlb.transform(movies.genre), columns = mlb.classes_ )

In [18]:
movies = pd.concat([movies, movies_genre], axis = 1)

In [19]:
movies['year']= movies.title.str.strip().apply(lambda x: int(x[-5:-1]))

In [20]:
movies.head()

,movieId,title,genre,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,year
0,1,Toy Story (1995),"[Animation, Children's, Comedy]",0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1995
1,2,Jumanji (1995),"[Adventure, Children's, Fantasy]",0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1995
2,3,Grumpier Old Men (1995),"[Comedy, Romance]",0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1995
3,4,Waiting to Exhale (1995),"[Comedy, Drama]",0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1995
4,5,Father of the Bride Part II (1995),[Comedy],0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1995


In [21]:
pd.get_dummies(users, columns = ['Gender'])

,UserID,Age,Occupation,Zip-code,Gender_F,Gender_M
0,1,1,10,48067,1,0
1,2,56,16,70072,0,1
2,3,25,15,55117,0,1
3,4,45,7,02460,0,1
4,5,25,20,55455,0,1
...,...,...,...,...,...,...
6035,6036,25,15,32603,1,0
6036,6037,45,1,76006,1,0
6037,6038,56,1,14706,1,0
6038,6039,45,0,01060,1,0


In [22]:
ratings['time']= pd.to_datetime(ratings.Timestamp, unit = "s")

In [23]:
ratings.head()

,UserID,MovieID,Rating,Timestamp,time
0,1,1193,5,978300760,2000-12-31 22:12:40
1,1,661,3,978302109,2000-12-31 22:35:09
2,1,914,3,978301968,2000-12-31 22:32:48
3,1,3408,4,978300275,2000-12-31 22:04:35
4,1,2355,5,978824291,2001-01-06 23:38:11


In [24]:
from datetime import datetime

In [25]:
ratings.time.min(), ratings.time.quantile(), ratings.time.max()

(Timestamp('2000-04-25 23:05:32'),
 Timestamp('2000-10-31 18:46:46'),
 Timestamp('2003-02-28 17:49:50'))

In [26]:
anchor_time = datetime(2000, 12,1 )

In [27]:
train = ratings[ratings.time < anchor_time]
test = ratings[ratings.time > anchor_time]

In [28]:
train.shape , test.shape

((791270, 5), (208939, 5))

In [29]:
train.head()

,UserID,MovieID,Rating,Timestamp,time
100409,670,3935,3,975628163,2000-11-30 23:49:23
100411,670,1252,4,975628353,2000-11-30 23:52:33
100412,670,720,4,975628314,2000-11-30 23:51:54
100415,670,3948,1,975628686,2000-11-30 23:58:06
100416,670,3949,4,975628670,2000-11-30 23:57:50


In [30]:
train.UserID.nunique()

5365

In [31]:
mean_rating_user = train.groupby('UserID').Rating.mean().reset_index()

In [32]:
mean_rating_user.columns=['UserID' , 'user_rating']

mean_rating_user

,UserID,user_rating
0,670,3.202532
1,671,4.384615
2,672,3.872340
3,673,3.126582
4,674,4.197183
...,...,...
5360,6036,3.302928
5361,6037,3.717822
5362,6038,3.800000
5363,6039,3.878049


In [33]:
mean_rating_movie = train.groupby('MovieID').Rating.mean().reset_index()

In [34]:
mean_rating_movie.columns=['MovieID' , 'movie_rating']
mean_rating_movie

,MovieID,movie_rating
0,1,4.147636
1,2,3.190731
2,3,3.005291
3,4,2.699248
4,5,3.000000
...,...,...
3657,3948,3.719101
3658,3949,4.337838
3659,3950,3.133333
3660,3951,3.950000


In [37]:
train = pd.merge(train, mean_rating_user, on='UserID' )

In [38]:
train = pd.merge(train, mean_rating_movie)

In [39]:
train.head()

,UserID,MovieID,Rating,Timestamp,time,user_rating,movie_rating
0,670,3935,3,975628163,2000-11-30 23:49:23,3.202532,2.846154
1,826,3935,3,975373258,2000-11-28 01:00:58,3.285714,2.846154
2,977,3935,3,975106391,2000-11-24 22:53:11,3.762500,2.846154
3,1017,3935,2,975009076,2000-11-23 19:51:16,3.251440,2.846154
4,1489,3935,2,975121808,2000-11-25 03:10:08,2.443763,2.846154


In [55]:
import numpy as np
from tensorflow import keras
! pip install pandas_profiling==2.7.1
import matplotlib.pyplot as plt
import pandas as pd 
from pandas_profiling import ProfileReport
from sklearn.model_selection import train_test_split

ImportError: ignored

In [56]:
x_train = train[['movie_rating', 'user_rating']]
x_train

,movie_rating,user_rating
0,2.846154,3.202532
1,2.846154,3.285714
2,2.846154,3.762500
3,2.846154,3.251440
4,2.846154,2.443763
...,...,...
791265,1.000000,3.053459
791266,5.000000,4.012346
791267,3.000000,3.436585
791268,4.000000,4.082192


In [57]:
y_train = train.Rating

In [58]:
from sklearn.linear_model import LinearRegression 

In [59]:
lr= LinearRegression()

In [66]:
lr.fit(x_train, y_train)

LinearRegression()

In [67]:
lr.score(x_train, y_train)

0.3289018745303842

In [68]:
y_hat = np.round(lr.predict(x_train))

In [69]:
from sklearn.metrics import r2_score

In [71]:
r2_score(y_train, y_hat)

0.26348298267411985